In [1]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [ ]:
@everywhere function Compute_Pi(N::Int)
    """
    Compute pi with a Monte Carlo simulation of N darts thrown in [-1,1]^2
    Returns estimate of pi
    """
    n_landed_in_circle = 0  # counts number of points that have radial coordinate < 1, i.e. in circle
    for i = 1:N
        x = rand() * 2 - 1  # uniformly distributed number on x-axis
        y = rand() * 2 - 1  # uniformly distributed number on y-axis

        r2 = x*x + y*y  # radius squared, in radial coordinates
        if r2 < 1.0
            n_landed_in_circle += 1
        end
    end

    return n_landed_in_circle / N * 4.0    
end

In [ ]:
Compute_Pi(100000)

In [ ]:
job = @spawn Compute_Pi(1000000000)
fetch(job)

In [ ]:
function Parallel_pi_computation(N::Int; ncores::Int=8)
    """
    Compute pi in parallel, over ncores cores, with a Monte Carlo simulation throwing N total darts
    """

    # compute sum of pi's estimated among all cores in parallel
    sum_of_pis = @parallel (+) for i=1:ncores
        Compute_Pi(ceil(Int, N / ncores))
    end

    return sum_of_pis / ncores  # average value
end

In [ ]:
@time Compute_Pi(8000000000)

In [ ]:
@time Parallel_pi_computation(8000000000)

In [ ]:
#Pkg.update()
#Pkg.add("PyPlot")
using PyPlot

In [ ]:
function incircle(r2)
    if r2<.25
        return true
    else
        return false
    end
end

In [ ]:
#The number of darts we will throw at the board.  We will see how accurate different numbers are
N=[10,25,50,75,100,250,500,750,1000,2500,5000,7500,10000];
# We will perform each number multiple times in order to calculate error bars
M=15;

In [ ]:
πapprox=zeros(Float64,length(N),M);

for ii in 1:length(N)
    #initialize an array of proper size
    X=zeros(Float64,N[ii],2);
    for jj in 1:M

        #popular our array with random numbers on the unit interval
        rand!(X);

        #calculate their radius squared
        R2=(X[:,1]-.5).^2+(X[:,2]-.5).^2

        # 4*number in circle / total number
        πapprox[ii,jj]=4.*length(filter(incircle,R2))/N[ii];

    end
end

In [ ]:
# Get our averages and standard deviations
πave=sum(πapprox,2)/M;
πstd=std(πapprox,2);

In [ ]:
title("Monte Carlo Estimation of π")
ylabel("π estimate")
xlabel("N points")
plot(N,π*ones(N));

for j in 1:M
    scatter(N,πapprox[:,j],marker="o",color="green");
end
ax=gca()
errorbar(N,π*ones(N),yerr=πstd,color="red",fmt="o")
ax[:set_xscale]("log");
ax[:set_xlim]([5,5*10^4]);

In [ ]:
title("Dependence of Monte Carlo Error on Number of Points")
ylabel("standard deviation")
xlabel("N points")
semilogx(N,πstd,marker="o");

In [ ]:
X=zeros(Float64,1000);
Y=zeros(Float64,1000);
rand!(X);
rand!(Y);
R2=(X-.5).^2+(Y-.5).^2;
Xc=[];
Yc=[]
for ii in 1:length(X)
    if R2[ii]<.25
        push!(Xc,X[ii]);
        push!(Yc,Y[ii]);
    end
end

title("The dartboard")
xlim(0,1)
ylim(0,1)
scatter(X,Y);
scatter(Xc,Yc,color="red");

In [ ]:
function MonteCarloPi(N::Int; ncores::Int=8)
    
    R=1;
    
    #first, we define a function that will check if a point is within the circle
    function Circle(R)
        if R<1.0
            return true
        else
            return false
        end
    end
    
    X=zeros(Float64,N,2);
    M=20;
    
    for jj in 1:M
        rand!(X);
        
        R=(X[:,1]).^2+(X[:,2]).^2

    end
   
    return 4.*length(filter(Circle,R))/N;
    
end

In [ ]:
@everywhere function MonteCarloPiCores(N::Int; ncores::Int=8)
    
    in_circle = 0;
    
    for i = 1 : N
        x = rand() * 2 - 1
        y = rand() * 2 - 1
        
        R = x*x + y*y;
        if R < 1.0
            in_circle += 1
        end
        
    end

    return in_circle/ N*4.0
    
end
    
#    #first, we define a function that will check if a point is within the circle
#    function Circle(R)
#        if R<1.0
#            return true
#        else
#            return false
#        end
#    end
#    
#    X=zeros(Float64,N,2);
#    M=20;
#    
#    for jj in 1:M
#        rand!(X);
#        
#        R=(X[:,1]).^2+(X[:,2]).^2
#
#    end
#   
#    return 4.*length(filter(Circle,R))/N;
#    
#end
#
#
#    n_landed_in_circle = 0  # counts number of points that have radial coordinate < 1, i.e. in circle
#        for i = 1:N
#            x = rand() * 2 - 1  # uniformly distributed number on x-axis
#            y = rand() * 2 - 1  # uniformly distributed number on y-axis
#
#            r2 = x*x + y*y  # radius squared, in radial coordinates
#            if r2 < 1.0
#                n_landed_in_circle += 1
#            end
#        end
#
#    return n_landed_in_circle / N * 4.0    
#end

In [ ]:
MonteCarloPiCores(10000)

In [ ]:
function Slices_of_Pi(N::Int; ncores::Int=8)
    """
    Compute pi in parallel, over ncores cores, with a Monte Carlo simulation throwing N total darts
    """

    # compute sum of pi's estimated among all cores in parallel
    sum_of_pis = @parallel (+) for i=1:ncores
        MonteCarloPiCores(ceil(Int, N / ncores))
    end

    return sum_of_pis / ncores  # average value
end

In [ ]:
Slices_of_Pi(1000000)

In [ ]:
@time MonteCarloPi(1000000)

In [ ]:
@time Slices_of_Pi(1000)

In [ ]:
rand()*2-1

In [4]:
function MC_Pi_1(N::Int)
    
    in_circle = 0;
    
    for i = 1 : N
        x = rand() * 2 - 1
        y = rand() * 2 - 1
        
        R = x*x + y*y;
        if R < 1.0
            in_circle += 1
        end
        
    end

    return in_circle/ N*4.0
    
end

MC_Pi_1 (generic function with 1 method)

In [15]:
@time MC_Pi_1(1000000000)

  9.083743 seconds (5 allocations: 176 bytes)


3.141552712

In [6]:
@everywhere function MC_Pi(N::Int; ncores::Int=4)
    
    in_circle = 0;
    
    for i = 1 : N
        x = rand() * 2 - 1
        y = rand() * 2 - 1
        
        R = x*x + y*y;
        if R < 1.0
            in_circle += 1
        end
        
    end

    return in_circle/ N*4.0
    
end


function Parallel_Pi(N::Int; ncores::Int=8)
    """
    Compute pi in parallel, over ncores cores, with a Monte Carlo simulation throwing N total darts
    """

    # compute sum of pi's estimated among all cores in parallel
    sum_of_pis = @parallel (+) for i=1:ncores
        MC_Pi(ceil(Int, N / ncores))
    end

    return sum_of_pis / ncores  # average value
end


Parallel_Pi (generic function with 1 method)

In [14]:
@time Parallel_Pi(1000000000)

  3.550851 seconds (764 allocations: 57.406 KB)


3.1416263840000003